In [1]:
#importing required libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
s = Service("C:\msedgedriver.exe")
driver = webdriver.Edge(service= s)
driver.get("https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=2022%2F23;trophy=89;type=season")
#page url we need to scrape
driver.maximize_window()

In [3]:
soup = BeautifulSoup(driver.page_source, 'lxml') #parsing the webpage

In [4]:
all_data = soup.find_all("tr",{"class":"data1"}) #class continer which contains all the required data

In [5]:
Match_id= []
Team1= []
Team2= []
for data in all_data:
    n= data.find_all("td",{"nowrap":"nowrap"})
    x = [i.text for i in n]
    Team1.append(x[0])
    Team2.append(x[1])
    Match_id.append(x[5])
#scraping match_id, Team1 and Team2    

In [6]:
teams=[]
for i in range(0,45):
    k= Team1[i]+" vs "+Team2[i]
    teams.append(k)   
#concatenating teams    

In [7]:
raw_bat_df=pd.DataFrame()
total_match_id = []
vs_teams = []
batting_position=[]

for i in range(0,45):  #iterating till the total number of matches played
    driver.find_element(By.XPATH, f'//*[@id="ciHomeContentlhs"]/div[3]/div/table[1]/tbody/tr[{i+1}]/td[7]/a').click()
    time.sleep(1)
    data= pd.read_html(f"{driver.current_url}")
    first_team_batter= data[0] #collecting batting data of the first team
    try:
        first_team_batter= first_team_batter[first_team_batter["BATTING"].isin(["Extras", "TOTAL"])==False] #cleaning data
        first_team_batter = first_team_batter[first_team_batter["BATTING"].str.startswith("Fall ")== False] #cleaning data
        first_team_batter = first_team_batter[first_team_batter["BATTING"].str.startswith("Did not bat")== False] #cleaning data
    except:
        pass
    second_team_batter = data[2] #collecting batting data of the second team
    try:
        second_team_batter= second_team_batter[second_team_batter["BATTING"].isin(["Extras", "TOTAL"])==False] #cleaning data
        second_team_batter = second_team_batter[second_team_batter["BATTING"].str.startswith("Fall ")== False] #cleaning data
        second_team_batter = second_team_batter[second_team_batter["BATTING"].str.startswith("Did not bat")== False] #cleaning data
        raw_bat_df= pd.concat([raw_bat_df,first_team_batter]) #joining dataframes
        raw_bat_df= pd.concat([raw_bat_df,second_team_batter]) #joining dataframes
        values= len(first_team_batter)+len(second_team_batter)
        batting_position.extend(list(range(1,len(first_team_batter)+1)))
        batting_position.extend(list(range(1,len(second_team_batter)+1)))    
        for value in range(values): #this for loop will add match_id and team(vs) to each row in the raw batting dataframe
            total_match_id.append(Match_id[i])
            vs_teams.append(teams[i])
    except:
        pass
    driver.back()
    time.sleep(1)

In [8]:
raw_bat_df.head(5)

,BATTING,Unnamed: 1,R,B,M,4s,6s,SR,Unnamed: 8,Unnamed: 9
0,Michael van Lingen,c Pramod Madushan b Chameera,3,6,7,0,0,50.00,NaN,NaN
2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,15,1,0,100.00,NaN,NaN
4,Jan Nicol Loftie-Eaton,c †Mendis b Karunaratne,20,12,18,1,2,166.66,NaN,NaN
6,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,49,2,0,108.33,NaN,NaN
8,Gerhard Erasmus (c),c Gunathilaka b PWH de Silva,20,24,30,0,0,83.33,NaN,NaN


In [9]:
raw_bat_df["Out/No Out"]= raw_bat_df["Unnamed: 1"].apply(lambda x: "Out" if x!= "not out" else "Not out") #adding out/not out column

In [10]:
raw_bat_df.insert(loc=0, column="Match_id",value= total_match_id, allow_duplicates= False)
raw_bat_df.insert(loc=1, column="Match",value= vs_teams, allow_duplicates= False)
raw_bat_df.insert(loc=2, column="Batting Position",value= batting_position, allow_duplicates= False)
#added new columns to raw batting dataframe

In [11]:
raw_bat_df.head(5)

,Match_id,Match,Batting Position,BATTING,Unnamed: 1,R,B,M,4s,6s,SR,Unnamed: 8,Unnamed: 9,Out/No Out
0,T20I # 1823,Namibia vs Sri Lanka,1,Michael van Lingen,c Pramod Madushan b Chameera,3,6,7,0,0,50.00,NaN,NaN,Out
2,T20I # 1823,Namibia vs Sri Lanka,2,Divan la Cock,c Shanaka b Pramod Madushan,9,9,15,1,0,100.00,NaN,NaN,Out
4,T20I # 1823,Namibia vs Sri Lanka,3,Jan Nicol Loftie-Eaton,c †Mendis b Karunaratne,20,12,18,1,2,166.66,NaN,NaN,Out
6,T20I # 1823,Namibia vs Sri Lanka,4,Stephan Baard,c DM de Silva b Pramod Madushan,26,24,49,2,0,108.33,NaN,NaN,Out
8,T20I # 1823,Namibia vs Sri Lanka,5,Gerhard Erasmus (c),c Gunathilaka b PWH de Silva,20,24,30,0,0,83.33,NaN,NaN,Out


In [12]:
raw_bat_df= raw_bat_df.drop(columns=["Unnamed: 1", "Unnamed: 8", "Unnamed: 9"])
#removed unwanted columns

In [13]:
raw_bat_df.head(5)

,Match_id,Match,Batting Position,BATTING,R,B,M,4s,6s,SR,Out/No Out
0,T20I # 1823,Namibia vs Sri Lanka,1,Michael van Lingen,3,6,7,0,0,50.00,Out
2,T20I # 1823,Namibia vs Sri Lanka,2,Divan la Cock,9,9,15,1,0,100.00,Out
4,T20I # 1823,Namibia vs Sri Lanka,3,Jan Nicol Loftie-Eaton,20,12,18,1,2,166.66,Out
6,T20I # 1823,Namibia vs Sri Lanka,4,Stephan Baard,26,24,49,2,0,108.33,Out
8,T20I # 1823,Namibia vs Sri Lanka,5,Gerhard Erasmus (c),20,24,30,0,0,83.33,Out


In [14]:
driver.close()

In [15]:
raw_bat_df.reset_index(drop=True, inplace=True)

In [16]:
raw_bat_df.rename({"BATTING": "Batsmen", "R": "Runs", "B": "Balls", "SR": "Strike Rate"}, axis=1, inplace= True)
#renamed columns

In [17]:
raw_bat_df.head(5)

,Match_id,Match,Batting Position,Batsmen,Runs,Balls,M,4s,6s,Strike Rate,Out/No Out
0,T20I # 1823,Namibia vs Sri Lanka,1,Michael van Lingen,3,6,7,0,0,50.00,Out
1,T20I # 1823,Namibia vs Sri Lanka,2,Divan la Cock,9,9,15,1,0,100.00,Out
2,T20I # 1823,Namibia vs Sri Lanka,3,Jan Nicol Loftie-Eaton,20,12,18,1,2,166.66,Out
3,T20I # 1823,Namibia vs Sri Lanka,4,Stephan Baard,26,24,49,2,0,108.33,Out
4,T20I # 1823,Namibia vs Sri Lanka,5,Gerhard Erasmus (c),20,24,30,0,0,83.33,Out


In [1]:
#raw_bat_df.to_csv("C:\\Users\\Gurjeet\\Desktop\\End to End Cricket Analytics Project\\Webscraped Files\\fact_batting.csv",encoding="utf-8-sig",index= False)

NameError: name 'raw_bat_df' is not defined